In [3]:
import pandas as pd
import numpy as np

Конвертирование данных в формат `Series` и `DataFrame`

In [10]:
np.random.seed(4)
values = [np.random.randint(-50, 50) for i in range(8)]
indices = [np.random.choice([1, 2, 3, 4, 5]) for i in range(8)]
print(f"{values  = }\n{indices = }")

values  = [-4, 5, 19, -49, 37, 22, 0, -41]
indices = [3, 5, 2, 1, 5, 3, 5, 3]


При конвертации данных в объекты классов `Series` или `DataFrame` производится индескирование строк. По умолчанию индексация уникальная и упорядоченная, начиная с 0:

In [11]:
s = pd.Series(values)
s

0    -4
1     5
2    19
3   -49
4    37
5    22
6     0
7   -41
dtype: int64

Индексация может быть произвольной в общем случае: индексы могут повторяться, идти не по порядку, и вовсе быть не числовыми. Если передать в конструктор класса `Series` последовательность значений в параметре `index`, то индексы строк будут сформированы из этой последовательности:

In [13]:
s_alt = pd.Series(data=values, index=indices)
s_alt

3    -4
5     5
2    19
1   -49
5    37
3    22
5     0
3   -41
dtype: int64

## Методы `loc` и `iloc`

Методы `loc` и `iloc` используются для выбора данных на определенных строках и колонках. Но есть принципиальная разница:

- `loc` выбирает строки и столбцы с определенными метками
- `iloc` выбирает строки и столбцы в определенных целочисленных позициях (`i-` означает *index*)

Методы `loc` и `iloc` принимают данные несколько непривычно, в квадратных скобках `[]`. Для понимания морфологии такой конструкции можно взглянуть на реализацию метода `b` в классе `A`:

In [12]:
class A:
    def __init__(self, lst):
        self.lst = lst
    def __getitem__(self, value):
        return self.lst[value]
    @property
    def b(self):
        return B(self)

class B:
    def __init__(self, lst):
        self.lst = lst
    def __getitem__(self, value):
        return self.lst[:value]

x = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
a = A(x)
print(f"{a[3] = }")
print(f"{a.b[3] = }")

a[3] = 'd'
a.b[3] = ['a', 'b', 'c']


### Выбор строк и колонок по индексам и именам

In [48]:
s_alt.iloc[2]   # выбирается третья по счету строка

19

In [17]:
type(s_alt.iloc[2])

numpy.int64

In [59]:
s_alt.loc[2]    # выбираются строки с индексами 2

2    -4
2    44
dtype: int64

In [60]:
s_alt.iloc[[1, 3]]

1     5
3   -49
dtype: int64

In [61]:
s_alt.loc[[1, 3]]

1     5
1     8
3   -49
3    22
dtype: int64

То же касается и объектов класса `DataFrame`.

In [62]:
df = pd.DataFrame({ 'team': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B'],
                    'points': [5, 7, 7, 9, 12, 9, 9, 4],
                    'assists': [11, 8, 10, 6, 6, 5, 9, 12]},
                    index=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
df

,team,points,assists
A,A,5,11
B,A,7,8
C,A,7,10
D,A,9,6
E,B,12,6
F,B,9,5
G,B,9,9
H,B,4,12


In [66]:
df.loc[['C', 'B']]

,team,points,assists
C,A,7,10
B,A,7,8


In [65]:
df.iloc[[2, 1]]

,team,points,assists
C,A,7,10
B,A,7,8


Тот же принцип применим и к колонкам.

In [67]:
df.loc[:, ['team', 'assists']]

,team,assists
A,A,11
B,A,8
C,A,10
D,A,6
E,B,6
F,B,5
G,B,9
H,B,12


In [70]:
df.iloc[:, [0, 2]]

,team,assists
A,A,11
B,A,8
C,A,10
D,A,6
E,B,6
F,B,5
G,B,9
H,B,12


Локализация и строк и колонок:

In [71]:
df.loc[['C', 'B'], ['team', 'assists']]

,team,assists
C,A,10
B,A,8


In [72]:
df.iloc[[2, 1], [0, 2]]

,team,assists
C,A,10
B,A,8


### Выбор строк по значениям признаков

In [76]:
df.loc[df['team'] == 'A']   # выбор строк по одному условию

,team,points,assists
A,A,5,11
B,A,7,8
C,A,7,10
D,A,9,6


In [82]:
# выбор строк, соответствующих нескольким условиям (И)
df.loc[(df['team'] == 'A') & (df['points'] == 7)]

,team,points,assists
B,A,7,8
C,A,7,10


In [89]:
print(type((df['team'] == 'A') & (df['points'] == 7)))
(df['team'] == 'A') & (df['points'] == 7)

<class 'pandas.core.series.Series'>


A    False
B     True
C     True
D    False
E    False
F    False
G    False
H    False
dtype: bool

In [90]:
# выбор строк, соответствующих одному из условий (ИЛИ)
df.loc[(df['team'] == 'A') | (df['assists'] == 6)]

,team,points,assists
A,A,5,11
B,A,7,8
C,A,7,10
D,A,9,6
E,B,12,6


## Создание новой колонки порождающей функцией

In [99]:
def f(row):
    return row["assists"] ** 2

df["squared_assists"] = df.apply(f, axis=1)
# либо при помощи аналогичной lambda функции
df["squared_assists"] = df.apply(lambda row : row["assists"] ** 2, axis=1)
df

,team,points,assists,squared_assists
A,A,5,11,121
B,A,7,8,64
C,A,7,10,100
D,A,9,6,36
E,B,12,6,36
F,B,9,5,25
G,B,9,9,81
H,B,4,12,144


## Исключение колонок или строк
Метод `isin[]` возвращает булевый массив NumPy, в соответствии с именами колонок, переданных в аргументе. Оператор `~` производит инвертирование массива. Используя такой массив как маску можно выбрать нужные данные методом `loc[]`.

In [119]:
import pandas as pd

# Create a sample DataFrame
data = {'col1': [1, 2, 3], 'col2': [4, 5, 6], 'col3': [7, 8, 9]}
df = pd.DataFrame(data)

# метод isin возвращает булевый массив в соответтвии 
print(f"{ df.columns.values }")
print(f"{ df.columns.isin(['col2', 'col3']) }")
print(f"{ ~df.columns.isin(['col2']) }")

# Select all columns except for 'col2':
print(f"{ df.loc[:, ~df.columns.isin(['col2'])] }")

['col1' 'col2' 'col3']
[False  True  True]
[ True False  True]
   col1  col3
0     1     7
1     2     8
2     3     9


Также можно удалить колонку и/или строку используя метод `drop()`. Еще один способ - получить множество (или список) имен колонок, например `set(df.columns)`, затем удалить ненужные имена и выбрать, используя оставшиеся имена.

In [124]:
print(df.drop(columns=['col2']))
print(df.drop(index=[1]))
print(df.drop(index=[1], columns=['col2']))

   col1  col3
0     1     7
1     2     8
2     3     9
   col1  col2  col3
0     1     4     7
2     3     6     9
   col1  col3
0     1     7
2     3     9
